In [21]:
import time
import re
from datetime import date
import math
import requests

import pandas as pd
from fuzzywuzzy import fuzz # we need fuzzy string matching for indentifying the same GPU
from pathlib import Path
import numpy as np
from tqdm import tqdm
from urllib.parse import quote
from bs4 import BeautifulSoup

# Get general spec (from TechPowerUP)

In [4]:
def memory_to_gb(mem_str):
    """Converts a string like "4 GB" to a float like 4.0 or "4 MB" to 0.004"""
    m = re.match("([.0-9]+)\s+([a-z]B)", mem_str, re.IGNORECASE)
    if m is None:
        return math.nan
    size, unit = float(m.groups()[0]), m.groups()[1]
    if unit.lower() == "gb":
        size = size
    elif unit.lower() == "mb":
        size = size / 1024.0
    elif unit.lower() == "kb":
        size = size / (1024.0 ** 2)
    return size

def fix_df_keys(df):
    """Fix some keys in the dataframe to make them more understandable."""
    df_new = dict(Manufacturer=[list(df.keys())[0][0] for _ in range(df.shape[-2])])
    for k in df.keys():
        key = k[1]
        df_new[key] = df[k]
    # memory, convert strings to floats representing GBs
    if "Memory" in df_new:
        memory = df_new["Memory"]
        split_memory = [[x.strip() for x in v.split(",")] for v in memory]
        split_memory = [x if len(x) == 3 else (*x, *(("",) * max(3 - len(x), 0))) for x in split_memory]
        msize, mtype, mwidth = zip(*split_memory)
        df_new["Memory Size (GB)"] = list(map(memory_to_gb, msize))
        df_new["Memory Type"] = list(mtype)
        df_new["Memory Bus Width"] = list(mwidth)
        del df_new["Memory"]

    # shader / TMUs / ROPs, split them into three columns
    if "Shaders / TMUs / ROPs" in df_new:
        keys = [x.strip() for x in "Shaders / TMUs / ROPs".split("/")]
        vals = zip(*[[x.strip() for x in v.split("/")] for v in df_new["Shaders / TMUs / ROPs"]])
        for (k, v) in zip(keys, vals):
            df_new[k] = v
        del df_new["Shaders / TMUs / ROPs"]
    return pd.DataFrame(df_new)

In [8]:
info_path = Path(".").absolute() / "data" / "partial_df_info.csv"
info_path.parent.mkdir(parents=True, exist_ok=True)
if info_path.exists():
    df_info = pd.read_csv(info_path)
else:
    URL = "https://www.techpowerup.com/gpu-specs/?mfgr=NVIDIA&released=REPLACE_YEAR&sort=name"
    dfs = dict()
    years = range(2023, 2008, -1)
    for year in tqdm(years):
        if year not in dfs:
            dfs[year] = pd.read_html(URL.replace("REPLACE_YEAR", str(year)))[1]
            time.sleep(5.0) # we have to be really slow to avoid being labeled as a bot
    df_info = pd.concat({year: fix_df_keys(df) for (year, df) in dfs.items()}.values())
    df_info.to_csv(info_path)

# CUDA Benchmark

In [9]:
benchmark_path = Path(".").absolute() / "data" / "partial_df_benchmark.csv"
benchmark_path.parent.mkdir(parents=True, exist_ok=True)
if benchmark_path.exists():
    df_benchmark = pd.read_csv(benchmark_path)
else:
    df_benchmark = pd.read_html("https://browser.geekbench.com/opencl-benchmarks")[0]
    df_benchmark.to_csv(benchmark_path)

In [10]:
def remove_words(word):
    """Remove kind of useless words in the GPU names: [nvidia, geforce]."""
    reps = ["nvidia", "geforce"]
    for rep in reps:
        word = re.sub(rep, "", word, flags=re.IGNORECASE).strip()
    return word
names_benchmark = list(map(remove_words, df_benchmark["Device"].tolist()))
names_info = list(map(remove_words, df_info["Product Name"].tolist()))

In [12]:
# this is CPU intensive
combined_path = Path(".").absolute() / "data" / "partial_df_combined.csv"
combined_path.parent.mkdir(parents=True, exist_ok=True)
if combined_path.exists():
    df_combined = pd.read_csv(combined_path)
else:
    scores = np.array([[fuzz.ratio(n1, n2) for n1 in names_benchmark] for n2 in names_info])
    best_info_idx = np.argmax(scores, -2)
    info_idx_to_keep = []
    benchmark_values = []
    for (i, (idx, name_benchmark)) in enumerate(zip(best_info_idx, names_benchmark)):
        score = scores[idx, i]
        if score > 90:
            if idx not in info_idx_to_keep:
                info_idx_to_keep.append(idx)
                benchmark_values.append(float(df_benchmark["Score"].values[i]))
    df_combined = df_info.iloc[info_idx_to_keep]
    df_combined["Score"] = benchmark_values
    df_combined.to_csv(combined_path)

/tmp/ipykernel_56396/1441540705.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined["Score"] = benchmark_values


# Get prices

In [14]:
def get_all_data(url, max_page=1):
    """We get prices data from a single, long ebay page with used listings."""
    output = set()
    for i in range(1, max_page + 1):
        t = time.time()
        data = requests.get(url + f"&_pgn={i}") if max_page > 1 else requests.get(url)
        #print(f"Request takes {time.time() - t:.4e} s")
        assert data.ok
        t = time.time()
        #content = BeautifulSoup(data.text, "html.parser")
        content = BeautifulSoup(data.text, "lxml")
        divs = content.find_all("li", attrs={"class": "s-item s-item__pl-on-bottom"})
        #print(f"Parsing takes {time.time() - t:.4e} s")
        k = 0
        for div in divs:
            price = div.find("span", attrs={"class": "s-item__price"})
            date = div.find("div", attrs={"class": "s-item__title--tagblock"})
            condition = div.find("span", attrs={"class": "SECONDARY_INFO"})
            if price is None:
                # print("Price is missing")
                continue
            if date is None:
                # print("Date is missing")
                continue
            if condition is None:
                # print("Condition is missing")
                continue
            if re.match("pre-owned", condition.text, flags=re.IGNORECASE) is None and re.match(
                "refurbished", condition.text, flags=re.IGNORECASE
            ) is None:
                #print(condition.text)
                continue
            try:
                price = float(price.text.replace("$", "").replace(",", ""))
            except ValueError:
                print(f"Could not parse price of {price.text}")
                continue
            match = re.match(r".*?([A-Za-z]+ [0-9]+, 2[0-9]+)", date.contents[0].text)
            if match is None:
                print(f"Could not match date against {date.contents[0]}")
                continue
            try:
                date = pd.to_datetime(match.groups()[0])
            except:
                print(f"Could not convert date {match.groups()[0]}")
            output.add((date, price))
            k += 1
        if k == 0:
            #print(f"Could not see any listings on page {i}")
            pass
    #print(f"Found {len(output)} listings")
    return output

def get_prices(query):
    """Getting prices for a particular GPU model."""
    url = f"https://www.ebay.com/sch/i.html?_from=R40&_nkw={quote(query)}&_sacat=0&LH_TitleDesc=0&_ipg=240&LH_Complete=1&LH_Sold=1&rt=nc&LH_ItemCondition=4"
    return [x[1] for x in get_all_data(url, max_page=1)]


def get_median_price(query):
    """Getting the median price for a particular GPU model with error handling."""
    try:
        prices = get_prices(query)
        assert len(prices) >= 3
    except AssertionError:
        return math.nan
    return np.median(prices)

In [17]:
# this takes and incredibly long time because we have to be careful not to trip ebay anti-scraping
with_price_path = Path(".").absolute() / "data" / "partial_df_with_price.csv"
with_price_path.parent.mkdir(parents=True, exist_ok=True)
if with_price_path.exists():
    df_with_price = pd.read_csv(with_price_path)
else:
    names = list(map(remove_words, df_combined["Product Name"].tolist()))
    prices = dict()
    for (i, name) in enumerate(tqdm(names)):
        if name not in prices or math.isnan(prices[name]):
            prices[name] = get_median_price(f"nvidia \"{name}\"")
        time.sleep(10.0)
    df_with_price = df_combined.copy()
    df_with_price["Price"] = list(prices.values())
    df_with_price["Score Per Price"] = df_with_price["Score"] / df_with_price["Price"]
    df_with_price.to_csv(with_price_path)

# Clean up the results

In [23]:
names = df_with_price["Product Name"].tolist()
names = list(map(lambda x: re.sub("[0-9]+ GB", "", x, flags=re.IGNORECASE).strip(), names))
names = list(map(lambda x: re.sub("geforce", "", x, flags=re.IGNORECASE).strip(), names))
df_final = df_with_price.copy()
df_final["Name"] = names
del df_final["Product Name"]
for key in df_final.keys():
    if re.match("Unnamed", key, flags=re.IGNORECASE):
        del df_final[key]
used_price_key = f"Used Price (eBay US) {date.today().strftime('%Y-%m-%d')}"
df_final[used_price_key] = df_final["Price"]
del df_final["Price"]
df_final["Performance Score"] = df_final["Score"]
del df_final["Score"]
cols = ["Name", "Memory Size (GB)", "Performance Score", used_price_key, "Score Per Price"]
df_final = df_final[cols + [x for x in df_final.keys() if x not in cols]]
df_final.to_csv(Path(".").absolute() / "data" / "nvidia_gpu_info.csv")

In [24]:
df_final

,Name,Memory Size (GB),Performance Score,Used Price (eBay US) 2023-03-15,Score Per Price,Manufacturer,GPU Chip,Released,Bus,GPU clock,Memory clock,Memory Type,Memory Bus Width,Shaders,TMUs,ROPs
0,RTX 4090,24.0,329909.0,1950.000,169.184103,NVIDIA,AD102,"Sep 20th, 2022",PCIe 4.0 x16,2235 MHz,1313 MHz,GDDR6X,384 bit,16384,512,176
1,RTX 4080,16.0,246342.0,1149.990,214.212297,NVIDIA,AD103,"Sep 20th, 2022",PCIe 4.0 x16,2205 MHz,1400 MHz,GDDR6X,256 bit,9728,304,112
2,RTX 4070 Ti,12.0,210475.0,NaN,NaN,NVIDIA,AD104,"Jan 3rd, 2023",PCIe 4.0 x16,2310 MHz,1313 MHz,GDDR6X,192 bit,7680,240,80
3,RTX 3090 Ti,24.0,209374.0,1049.980,199.407608,NVIDIA,GA102,"Jan 27th, 2022",PCIe 4.0 x16,1560 MHz,1313 MHz,GDDR6X,384 bit,10752,336,112
4,RTX 3080 Ti,12.0,209081.0,749.995,278.776525,NVIDIA,GA102,"May 31st, 2021",PCIe 4.0 x16,1365 MHz,1188 MHz,GDDR6X,384 bit,10240,320,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,610M,1.0,1086.0,36.790,29.518891,NVIDIA,GF117B,"Dec 1st, 2011",PCIe 2.0 x16,625 MHz,900 MHz,DDR3,64 bit,48,8,8
220,GT 520MX,1.0,1081.0,NaN,NaN,NVIDIA,GF119B,"May 30th, 2011",PCIe 2.0 x16,900 MHz,900 MHz,DDR3,64 bit,48,8,4
221,NVS 310,0.5,851.0,14.990,56.771181,NVIDIA,GF119,"Jun 26th, 2012",PCIe 2.0 x16,523 MHz,875 MHz,DDR3,64 bit,48,8,4
222,NVS 315,1.0,851.0,12.990,65.511932,NVIDIA,GF119,"Mar 10th, 2013",PCIe 2.0 x16,523 MHz,875 MHz,DDR3,64 bit,48,8,4
